In [1]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from scipy.signal import find_peaks
import os
from src.data_utils import get_list_of_data_testing, search_in_list

Datos Lunares

In [2]:

Directorio = 'data/lunar/test/data/'

# Listar todas las carpetas dentro del directorio
SismosLuna = pd.DataFrame()

# Iterar sobre cada carpeta
csv_paths = get_list_of_data_testing("lunar", "csv")["path"].to_list()
mseed_paths = get_list_of_data_testing("lunar", "mseed")["path"].to_list()

# Iterar sobre los archivos CSV y mseed al mismo tiempo
for ruta_csv in csv_paths:
    archivo_csv = ruta_csv.split("/")[-1]
    base_name = ".".join(ruta_csv.split("/")[-1].split(".")[:-1])
    ruta_mseed = mseed_paths[search_in_list(base_name, mseed_paths)]
    
    # Leer archivo CSV
    DataTraining = pd.read_csv(ruta_csv)
    print(f'Leyendo archivo CSV: {ruta_csv}')
    
    # Leer archivo mseed
    st = read(ruta_mseed)
    print(f'Leyendo archivo mseed: {ruta_mseed}')


    # This is how you get the data and the time, which is in seconds
    tr = st.traces[0].copy()
    tr_times = tr.times()
    tr_data = tr.data


    print("Reducción de ruido por filtro high-pass, band-pass")
    # Definir la frecuencia de corte para el filtro high-pass
    cutoff_freq = 3  # Ajusta este valor según sea necesario

    # Crear una copia del objeto Stream
    st_filt_hp = st.copy()

    # Aplicar un filtro high-pass
    st_filt_hp.filter('highpass', freq=cutoff_freq)


    # Set the minimum frequency
    minfreq = 0.5
    maxfreq = 1

    # Going to create a separate trace for the filter data
    st_filt = st_filt_hp.copy()
    st_filt.filter('bandpass',freqmin=minfreq,freqmax=maxfreq)
    tr_filt = st_filt.traces[0].copy()
    tr_times_filt = tr_filt.times()
    tr_data_filt = tr_filt.data
    print("Finaliza reducción")

    print("Ejecutando la transformada de hilbert")
    analytical_signal = hilbert(tr_data_filt)
    
    amplitude_envelope = np.abs(analytical_signal)
    print("Transformada de hilbert finalizada")


    print("Iniciando detección de sismos")
    # Detectar picos en la energía
    # Ajustar el umbral de acuerdo a tus datos
    min_distance = 40000
    threshold = np.percentile(amplitude_envelope, 99.5)  # Ajustar percentil si es necesario
    peaks, properties = find_peaks(amplitude_envelope, height=threshold, distance=min_distance)

    # Definir ventana para acumular energía alrededor de cada pico
    window_size = 5000  # Puedes ajustar este tamaño según tus datos

    # Calcular la energía acumulada en la vecindad de cada pico
    accumulated_energy = []
    for peak in peaks:
        left = max(0, peak - window_size)
        right = min(len(amplitude_envelope), peak + window_size)
        energy_sum = np.sum(amplitude_envelope[left:right])
        accumulated_energy.append(energy_sum)

    # Seleccionar el pico con mayor energía acumulada
    max_energy_peak_idx = np.argmax(accumulated_energy)
    best_peak = peaks[max_energy_peak_idx]
    print("Detección de sismos finalizada")

    print("Exportando los valores del rango del sismo a una carpeta")
    # Encontrar los índices donde la envolvente supera el umbral
    above_threshold = amplitude_envelope > threshold
    RangoSismo = DataTraining.iloc[above_threshold]


    output_dir = 'output/data_rango_sismo/'  # Directorio donde se guardarán los archivos

    # Crear el directorio de salida si no existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Definir un nombre único para el archivo basado en la carpeta y archivo original
    nombre_salida = f'DataRangoSismo_{ruta_csv.split("/")[-2]}_{archivo_csv[:-4]}.csv'
    ruta_salida = os.path.join(output_dir, nombre_salida)
    
    # Guardar el archivo CSV
    RangoSismo.to_csv(ruta_salida, index=False)
    print(f'Archivo guardado en: {ruta_salida}')

    PicoSismo = DataTraining.iloc[[best_peak]]

    # Agregar las filas al DataFrame consolidado usando pd.concat
    PicoSismo["filename"] = [base_name]
    SismosLuna = pd.concat([SismosLuna, PicoSismo], ignore_index=True)

SismosLuna.to_csv('Sismos_Luna.csv', index=False)

Leyendo archivo CSV: data/lunar/test/data/S16_GradeA/xa.s16.00.mhz.1972-11-08HR00_evid00080.csv
Leyendo archivo mseed: data/lunar/test/data/S16_GradeA/xa.s16.00.mhz.1972-11-08HR00_evid00080.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en: output/data_rango_sismo/DataRangoSismo_S16_GradeA_xa.s16.00.mhz.1972-11-08HR00_evid00080.csv


/tmp/ipykernel_29378/844006441.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PicoSismo["filename"] = [base_name]


Leyendo archivo CSV: data/lunar/test/data/S16_GradeA/xa.s16.00.mhz.1974-12-12HR02_evid00168.csv
Leyendo archivo mseed: data/lunar/test/data/S16_GradeA/xa.s16.00.mhz.1974-12-12HR02_evid00168.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en: output/data_rango_sismo/DataRangoSismo_S16_GradeA_xa.s16.00.mhz.1974-12-12HR02_evid00168.csv


/tmp/ipykernel_29378/844006441.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PicoSismo["filename"] = [base_name]


Leyendo archivo CSV: data/lunar/test/data/S16_GradeA/xa.s16.00.mhz.1974-12-15HR00_evid00172.csv
Leyendo archivo mseed: data/lunar/test/data/S16_GradeA/xa.s16.00.mhz.1974-12-15HR00_evid00172.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en: output/data_rango_sismo/DataRangoSismo_S16_GradeA_xa.s16.00.mhz.1974-12-15HR00_evid00172.csv


...

/tmp/ipykernel_29378/844006441.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PicoSismo["filename"] = [base_name]


Leyendo archivo CSV: data/lunar/test/data/S16_GradeB/xa.s16.00.mhz.1973-12-18HR00_evid00487.csv
Leyendo archivo mseed: data/lunar/test/data/S16_GradeB/xa.s16.00.mhz.1973-12-18HR00_evid00487.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en: output/data_rango_sismo/DataRangoSismo_S16_GradeB_xa.s16.00.mhz.1973-12-18HR00_evid00487.csv


/tmp/ipykernel_29378/844006441.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PicoSismo["filename"] = [base_name]


Leyendo archivo CSV: data/lunar/test/data/S16_GradeB/xa.s16.00.mhz.1974-11-14HR00_evid00587.csv
Leyendo archivo mseed: data/lunar/test/data/S16_GradeB/xa.s16.00.mhz.1974-11-14HR00_evid00587.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en: output/data_rango_sismo/DataRangoSismo_S16_GradeB_xa.s16.00.mhz.1974-11-14HR00_evid00587.csv


/tmp/ipykernel_29378/844006441.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PicoSismo["filename"] = [base_name]


In [3]:
SismosLuna.head()

,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),velocity(m/s),filename
0,1972-11-08T18:51:41.932962,67901.433962,-3.581083e-09,xa.s16.00.mhz.1972-11-08HR00_evid00080
1,1974-12-12T17:49:14.702460,56031.245283,-6.409832e-11,xa.s16.00.mhz.1974-12-12HR02_evid00168
2,1974-12-15T09:14:54.254849,33293.735849,-2.701294e-09,xa.s16.00.mhz.1974-12-15HR00_evid00172
3,1973-07-31T10:02:26.188377,36145.660377,3.743215e-09,xa.s16.00.mhz.1973-07-31HR00_evid00123
4,1972-11-06T04:31:20.353057,16279.849057,-7.014655e-11,xa.s16.00.mhz.1972-11-06HR00_evid00079


Datos de Marte

In [5]:
Directorio = 'data/mars/test/data/'




archivos = os.listdir(Directorio)
SismosMarte = pd.DataFrame()
# Filtrar archivos que sean CSV y mseed

# Iterar sobre cada carpeta
csv_paths = get_list_of_data_testing("mars", "csv")["path"].to_list()
mseed_paths = get_list_of_data_testing("mars", "mseed")["path"].to_list()

# Iterar sobre los archivos CSV y mseed al mismo tiempo
for ruta_csv in csv_paths:
    archivo_csv = ruta_csv.split("/")[-1]
    base_name = ".".join(ruta_csv.split("/")[-1].split(".")[:-1])
    ruta_mseed = mseed_paths[search_in_list(base_name, mseed_paths)]

    # Leer archivo CSV
    DataTraining = pd.read_csv(ruta_csv)
    print(f'Leyendo archivo CSV: {ruta_csv}')
        
    # Leer archivo mseed
    st = read(ruta_mseed)
    print(f'Leyendo archivo mseed: {ruta_mseed}')


    # This is how you get the data and the time, which is in seconds
    tr = st.traces[0].copy()
    tr_times = tr.times()
    tr_data = tr.data


    print("Reducción de ruido por filtro high-pass, band-pass")
    # Definir la frecuencia de corte para el filtro high-pass
    cutoff_freq = 3  # Ajusta este valor según sea necesario

    # Crear una copia del objeto Stream
    st_filt_hp = st.copy()

    # Aplicar un filtro high-pass
    st_filt_hp.filter('highpass', freq=cutoff_freq)


    # Set the minimum frequency
    minfreq = 0.5
    maxfreq = 1

    # Going to create a separate trace for the filter data
    st_filt = st_filt_hp.copy()
    st_filt.filter('bandpass',freqmin=minfreq,freqmax=maxfreq)
    tr_filt = st_filt.traces[0].copy()
    tr_times_filt = tr_filt.times()
    tr_data_filt = tr_filt.data
    print("Finaliza reducción")


    print("Ejecutando la transformada de hilbert")
    analytical_signal = hilbert(tr_data_filt)
    amplitude_envelope = np.abs(analytical_signal)
    print("Transformada de hilbert finalizada")


    print("Iniciando detección de sismos")
    # Detectar picos en la energía
    # Ajustar el umbral de acuerdo a tus datos
    min_distance = 40000
    threshold = np.percentile(amplitude_envelope, 99.5)  # Ajustar percentil si es necesario
    peaks, properties = find_peaks(amplitude_envelope, height=threshold, distance=min_distance)

    # Definir ventana para acumular energía alrededor de cada pico
    window_size = 5000  # Puedes ajustar este tamaño según tus datos

    # Calcular la energía acumulada en la vecindad de cada pico
    accumulated_energy = []
    for peak in peaks:
        left = max(0, peak - window_size)
        right = min(len(amplitude_envelope), peak + window_size)
        energy_sum = np.sum(amplitude_envelope[left:right])
        accumulated_energy.append(energy_sum)

    # Seleccionar el pico con mayor energía acumulada
    max_energy_peak_idx = np.argmax(accumulated_energy)
    best_peak = peaks[max_energy_peak_idx]
    print("Detección de sismos finalizada")

    print("Exportando los valores del rango del sismo a una carpeta")
    # Encontrar los índices donde la envolvente supera el umbral
    above_threshold = amplitude_envelope > threshold
    RangoSismoMarte = DataTraining.iloc[above_threshold]


    output_dir = 'output/data_rango_sismo_Marte/'  # Directorio donde se guardarán los archivos

    # Crear el directorio de salida si no existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Definir un nombre único para el archivo basado en la carpeta y archivo original
    nombre_salida = f'DataRangoSismo_{archivo_csv[:-4]}.csv'
    ruta_salida = os.path.join(output_dir, nombre_salida)
        
    # Guardar el archivo CSV
    RangoSismoMarte.to_csv(ruta_salida, index=False)
    print(f'Archivo guardado en: {ruta_salida}')

    PicoSismo = DataTraining.iloc[[best_peak]]
    PicoSismo["filename"] = [base_name]

    # Agregar las filas al DataFrame consolidado usando pd.concat
    SismosMarte = pd.concat([SismosMarte, PicoSismo], ignore_index=True)

SismosMarte.to_csv('Sismos_Marte.csv', index=False)

Leyendo archivo CSV: data/mars/test/data/XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv
Leyendo archivo mseed: data/mars/test/data/XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en: output/data_rango_sismo_Marte/DataRangoSismo_XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv
Leyendo archivo CSV: data/mars/test/data/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv
Leyendo archivo mseed: data/mars/test/data/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en:

/tmp/ipykernel_29378/2009510571.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PicoSismo["filename"] = [base_name]
/tmp/ipykernel_29378/2009510571.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PicoSismo["filename"] = [base_name]
/tmp/ipykernel_29378/2009510571.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

Leyendo archivo CSV: data/mars/test/data/XB.ELYSE.02.BHV.2021-05-02HR01_evid0017.csv
Leyendo archivo mseed: data/mars/test/data/XB.ELYSE.02.BHV.2021-05-02HR01_evid0017.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en: output/data_rango_sismo_Marte/DataRangoSismo_XB.ELYSE.02.BHV.2021-05-02HR01_evid0017.csv
Leyendo archivo CSV: data/mars/test/data/XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.csv
Leyendo archivo mseed: data/mars/test/data/XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en:

/tmp/ipykernel_29378/2009510571.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PicoSismo["filename"] = [base_name]
/tmp/ipykernel_29378/2009510571.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PicoSismo["filename"] = [base_name]
/tmp/ipykernel_29378/2009510571.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en: output/data_rango_sismo_Marte/DataRangoSismo_XB.ELYSE.02.BHV.2022-04-09HR22_evid0002.csv
Leyendo archivo CSV: data/mars/test/data/XB.ELYSE.02.BHV.2019-09-21HR03_evid0032.csv
Leyendo archivo mseed: data/mars/test/data/XB.ELYSE.02.BHV.2019-09-21HR03_evid0032.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en: output/data_rango_sismo_Marte/DataRangoSismo_XB.ELYSE.02.BHV.2019-09-21HR03_evid0032.csv
Leyendo archivo CSV: data/mars/test/data/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv
Leyendo archivo mseed: data/mars/test/data/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la

/tmp/ipykernel_29378/2009510571.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PicoSismo["filename"] = [base_name]
/tmp/ipykernel_29378/2009510571.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PicoSismo["filename"] = [base_name]
/tmp/ipykernel_29378/2009510571.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [ ]:
best_peak